In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
with open('tasks_top10_mapped_parameters.pkl', 'rb') as input_file:
    tasks_top10_mapped_parameters = pickle.load(input_file)   
tasks_top10_mapped_parameters.shape

(10396, 9)

<h4>Build AST</h4>

In [3]:
from build_sequence_tokens import return_tokens, rem_nested_lists,convert_int_bool_to_str,tokenize_task,tokenize_name,is_empty

In [4]:
mapped10 = tasks_top10_mapped_parameters.copy()

In [5]:
mapped10['has_params'] = mapped10['found_used_parameters'].apply(lambda x: is_empty(x['intersected_params']))
mapped10['has_params'][0]

True

In [6]:
mapped10['initial_ast'] = mapped10['method_description'].apply(lambda x: return_tokens(x))
mapped10['second_ast'] = mapped10['initial_ast'].apply(lambda x: convert_int_bool_to_str(x))
mapped10['third_ast'] = mapped10['second_ast'].apply(lambda x: rem_nested_lists(x))

In [9]:
print(mapped10['method_description'][16331])
mapped10['third_ast'][16331]

{'command': 'yum-config-manager --enable {{ item }}', 'with_items': ['rhui-REGION-rhel-server-rhscl', 'rhel-server-rhscl-6-rpms'], 'when': ["ansible_distribution == 'RedHat' and  ansible_distribution_major_version == '6'", "wazuh_manager_config.cluster.disable != 'yes'"]}


['AnsibleTask',
 'command',
 'yum-config-manager --enable {{ item }}',
 'with_items',
 'rhui-REGION-rhel-server-rhscl,rhel-server-rhscl-6-rpms',
 'when',
 "ansible_distribution == 'RedHat' and  ansible_distribution_major_version == '6',wazuh_manager_config.cluster.disable != 'yes'"]

<h4>Tokenize</h4>

In [10]:
from nltk import word_tokenize

In [11]:
task_name_tokens = [word_tokenize(token) for token in mapped10['task_name']]

In [12]:
task_name_tokens[0]

['restart', 'datadog-agent']

In [13]:
mapped10['first_tokens'] = mapped10['third_ast'].apply(lambda x: tokenize_task(x))

In [14]:
mapped10 = mapped10.reset_index(drop=True)

In [17]:
m10 = mapped10.copy()

In [22]:
from pre_process import remove_symbols, remove_symbols_simple,flatten_list

In [23]:
m10['second_tokens'] = m10['first_tokens'].apply(lambda x: remove_symbols(x))
m10['third_tokens'] = m10['second_tokens'].apply(lambda x: flatten_list(x))

In [24]:
print(m10['method_description'][0])
m10['third_tokens'][0]

{'service': {'name': 'datadog-agent', 'state': 'restarted'}, 'when': 'datadog_enabled and not ansible_check_mode and not ansible_os_family == "Windows"'}


['AnsibleTask',
 'service',
 'name',
 'datadog-agent',
 'state',
 'restarted',
 'when',
 'datadog_enabled',
 'and',
 'not',
 'ansible_check_mode',
 'and',
 'not',
 'ansible_os_family',
 'Windows']

In [25]:
m10['token_task_names'] = task_name_tokens

In [26]:
m10 = m10.drop(columns=['initial_ast', 'second_ast','first_tokens','second_tokens'])

In [28]:
m10['descr_one_string'] = m10['third_tokens'].apply(lambda x: ' '.join(x))

In [29]:
m10.shape

(10396, 14)

In [30]:
with open('tasks_top10_ast_tokenization_sodalite.pkl', 'wb') as output_file:
    pickle.dump(m10, output_file)

In [ ]:
with open('tasks_top10_ast_tokenization_sodalite.pkl', 'rb') as input_file:
    mapped10_ast_token = pickle.load(input_file)   
mapped10_ast_token.shape